In [8]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor
import requests

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="/data/vdc/tangzichen/qwen2.5",
    # local_dir_use_symlinks=False,
    # torch_dtype="auto",
    # device_map="cuda:1"
)


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/data/vdc/tangzichen/qwen2.5")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

text


'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n'

In [4]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

model_inputs['input_ids']

tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,  35127,    752,    264,
           2805,  16800,    311,   3460,   4128,   1614,     13, 151645,    198,
         151644,  77091,    198]])

In [5]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

In [6]:
# 开头一些token是输入, 需要去除
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
generated_ids

[tensor([ 39814,      0,    362,   3460,   4128,   1614,    320,   4086,     44,
              8,    374,    264,    943,    315,  20443,  11229,    429,    646,
           6923,   3738,  12681,   1467,   3118,    389,   1946,    821,    323,
           4862,     13,   4220,   4119,    525,   6188,    311,    387,   2952,
            311,   3535,   6351,   5810,  15459,    323,   6923,  55787,    323,
           2266,   1832,   9760,  14507,    382,   3862,    525,   2155,   4494,
            315,    444,  10994,     82,     11,   2670,   5538,   6832,   5980,
           4119,   1075,  86870,     11,  64074,  29728,  14155,     11,    323,
           1766,   1388,  28411,  42721,  14155,     11,    438,   1632,    438,
            803,   8606,  19827,   1741,    438,   5912,   5980,   5942,    476,
          28464,   4119,     13,    576,   1429,  11245,    444,  10994,     82,
           2924,    279,    479,   2828,   4013,    504,  16580,  43936,     11,
            892,    702,   1

In [7]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

"Sure! A large language model (LLM) is a type of artificial intelligence that can generate human-like text based on input data and training. These models are designed to be able to understand complex natural languages and generate coherent and contextually relevant responses.\n\nThere are different types of LLMs, including deep learning-based models like transformers, recurrent neural networks, and generative adversarial networks, as well as more traditional approaches such as rule-based systems or statistical models. The most famous LLMs include the GPT series from Anthropic, which has been used in various applications, and the BERT series from Alibaba Cloud, which was developed for text generation tasks.\n\nLarge language models have the potential to revolutionize many industries, including healthcare, finance, legal services, and education. They can help automate some tasks and improve the quality of information provided by humans. However, it's important to note that the use of LLM

In [9]:
processor = AutoProcessor.from_pretrained('google/siglip-base-patch16-224', cache_dir="/data/vdc/tangzichen/siglip")

/home/tangzichen.tzc/anaconda3/envs/humangaussian/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

texts = ["a photo of 2 cats", "a photo of 2 dogs"]

inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

inputs

NameError: name 'Image' is not defined